<a href="https://colab.research.google.com/github/Janebhop/fraud-detection-course/blob/main/notebook/chapter3/Chapter3_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part Model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Import Data

In [ ]:
!gdown --id 1VNr0ywELbmU2JApGmL0PrAnrEh3VHcgW

Downloading...
From: https://drive.google.com/uc?id=1VNr0ywELbmU2JApGmL0PrAnrEh3VHcgW
To: /content/PS_20174392719_1491204439457_log.csv
100% 494M/494M [00:02<00:00, 180MB/s]


In [ ]:
dataPath = '/content/PS_20174392719_1491204439457_log.csv'

In [ ]:
df_test = pd.read_csv(dataPath, nrows=10)

float_cols = [c for c in df_test if df_test[c].dtype == "float64"]
float32_cols = {c: np.float32 for c in float_cols}

In [ ]:
df_test

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.0,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.0,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.0,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.0,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.0,0.00,0,0
5,1,PAYMENT,7817.71,C90045638,53860.00,46042.29,M573487274,0.0,0.00,0,0
6,1,PAYMENT,7107.77,C154988899,183195.00,176087.23,M408069119,0.0,0.00,0,0
7,1,PAYMENT,7861.64,C1912850431,176087.23,168225.59,M633326333,0.0,0.00,0,0
8,1,PAYMENT,4024.36,C1265012928,2671.00,0.00,M1176932104,0.0,0.00,0,0
9,1,DEBIT,5337.77,C712410124,41720.00,36382.23,C195600860,41898.0,40348.79,0,0


In [ ]:
data = pd.read_csv(dataPath, engine='c', dtype=float32_cols)
data

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9.839640e+03,C1231006815,170136.000,160296.359375,M1979787155,0.000000e+00,0.000,0,0
1,1,PAYMENT,1.864280e+03,C1666544295,21249.000,19384.720703,M2044282225,0.000000e+00,0.000,0,0
2,1,TRANSFER,1.810000e+02,C1305486145,181.000,0.000000,C553264065,0.000000e+00,0.000,1,0
3,1,CASH_OUT,1.810000e+02,C840083671,181.000,0.000000,C38997010,2.118200e+04,0.000,1,0
4,1,PAYMENT,1.166814e+04,C2048537720,41554.000,29885.859375,M1230701703,0.000000e+00,0.000,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,3.396821e+05,C786484425,339682.125,0.000000,C776919290,0.000000e+00,339682.125,1,0
6362616,743,TRANSFER,6.311410e+06,C1529008245,6311409.500,0.000000,C1881841831,0.000000e+00,0.000,1,0
6362617,743,CASH_OUT,6.311410e+06,C1162922333,6311409.500,0.000000,C1365125890,6.848884e+04,6379898.000,1,0
6362618,743,TRANSFER,8.500025e+05,C1685995037,850002.500,0.000000,C2080388513,0.000000e+00,0.000,1,0


### Preprocessing

In [ ]:
# cleaning data 
# checking null values

data.isnull().values.any()

False

In [ ]:
# feature selection
data = data.drop(['nameOrig','nameDest'],axis = 1)

In [ ]:
# feature engineering

dummy_type = pd.get_dummies(data['type'])
dummy_type

,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,0,1
3,0,1,0,0,0
4,0,0,0,1,0
...,...,...,...,...,...
6362615,0,1,0,0,0
6362616,0,0,0,0,1
6362617,0,1,0,0,0
6362618,0,0,0,0,1


In [ ]:
data = (pd.concat([data,dummy_type], axis = 1).reset_index(drop = True)).drop(['type'],axis = 1)[['step','CASH_IN','CASH_OUT','DEBIT','PAYMENT','TRANSFER','amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest','isFlaggedFraud','isFraud']]
data

,step,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,isFraud
0,1,0,0,0,1,0,9.839640e+03,170136.000,160296.359375,0.000000e+00,0.000,0,0
1,1,0,0,0,1,0,1.864280e+03,21249.000,19384.720703,0.000000e+00,0.000,0,0
2,1,0,0,0,0,1,1.810000e+02,181.000,0.000000,0.000000e+00,0.000,0,1
3,1,0,1,0,0,0,1.810000e+02,181.000,0.000000,2.118200e+04,0.000,0,1
4,1,0,0,0,1,0,1.166814e+04,41554.000,29885.859375,0.000000e+00,0.000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,0,1,0,0,0,3.396821e+05,339682.125,0.000000,0.000000e+00,339682.125,0,1
6362616,743,0,0,0,0,1,6.311410e+06,6311409.500,0.000000,0.000000e+00,0.000,0,1
6362617,743,0,1,0,0,0,6.311410e+06,6311409.500,0.000000,6.848884e+04,6379898.000,0,1
6362618,743,0,0,0,0,1,8.500025e+05,850002.500,0.000000,0.000000e+00,0.000,0,1


In [ ]:
data['errorBalanceOrig'] = abs(data['newbalanceOrig'] - data['oldbalanceOrg']) - data['amount'] 
data['errorBalanceDest'] = abs(data['oldbalanceDest'] - data['newbalanceDest']) - data['amount'] 
data

,step,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,isFraud,errorBalanceOrig,errorBalanceDest
0,1,0,0,0,1,0,9.839640e+03,170136.000,160296.359375,0.000000e+00,0.000,0,0,0.000977,-9.839640e+03
1,1,0,0,0,1,0,1.864280e+03,21249.000,19384.720703,0.000000e+00,0.000,0,0,-0.000732,-1.864280e+03
2,1,0,0,0,0,1,1.810000e+02,181.000,0.000000,0.000000e+00,0.000,0,1,0.000000,-1.810000e+02
3,1,0,1,0,0,0,1.810000e+02,181.000,0.000000,2.118200e+04,0.000,0,1,0.000000,2.100100e+04
4,1,0,0,0,1,0,1.166814e+04,41554.000,29885.859375,0.000000e+00,0.000,0,0,0.000977,-1.166814e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,0,1,0,0,0,3.396821e+05,339682.125,0.000000,0.000000e+00,339682.125,0,1,0.000000,0.000000e+00
6362616,743,0,0,0,0,1,6.311410e+06,6311409.500,0.000000,0.000000e+00,0.000,0,1,0.000000,-6.311410e+06
6362617,743,0,1,0,0,0,6.311410e+06,6311409.500,0.000000,6.848884e+04,6379898.000,0,1,0.000000,-5.000000e-01
6362618,743,0,0,0,0,1,8.500025e+05,850002.500,0.000000,0.000000e+00,0.000,0,1,0.000000,-8.500025e+05


In [ ]:
# splitting data

# fraud and no fraud transactions are separated by 'isFraud' column
dataF = data[data['isFraud'] == 1]
dataNF = data[data['isFraud'] == 0]

# split fraud transactions into 80 percent and 20 percent sessions
mskF = np.random.rand(len(dataF)) < 0.8
trainF = dataF[mskF]
testF = dataF[~mskF]

# split no fraud transactions into 80 percent and 20 percent sessions
mskNF = np.random.rand(len(dataNF)) < 0.8
trainNF = dataNF[mskNF]
testNF = dataNF[~mskNF]

# create training data and testing data
train_data = pd.concat([trainF,trainNF]).reset_index(drop = True)
test_data = pd.concat([testF,testNF]).reset_index(drop = True)

#create input data attributes by dropped irrevalent and target column 
train_feature = train_data.drop(['isFraud'],axis = 1)
test_feature = test_data.drop(['isFraud'],axis = 1)

train_label = train_data['isFraud']
test_label = test_data['isFraud']

In [ ]:
# feature scaling

scaler = preprocessing.StandardScaler()
train_feature_scaled =scaler.fit_transform(train_feature)

test_feature_scaled = scaler.transform(test_feature)

In [ ]:
col = ['step', 'CASH_IN', 'CASH_OUT', 'DEBIT', 'PAYMENT', 'TRANSFER', 'amount',
       'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest',
       'isFlaggedFraud', 'errorBalanceOrig', 'errorBalanceDest']

In [ ]:
data_feature_train = pd.DataFrame(data=train_feature_scaled, columns=col)
data_feature_train

,step,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,errorBalanceOrig,errorBalanceDest
0,-1.702315,-0.531236,-0.736018,-0.081037,-0.715003,3.308342,-0.297684,-0.288732,-0.292522,-0.323974,-0.333484,-0.00147,0.213350,-0.015097
1,-1.702315,-0.531236,1.358662,-0.081037,-0.715003,-0.302266,-0.297684,-0.288732,-0.292522,-0.317736,-0.333484,-0.00147,0.213350,0.036982
2,-1.702315,-0.531236,-0.736018,-0.081037,-0.715003,3.308342,-0.293333,-0.287823,-0.292522,-0.323974,-0.333484,-0.00147,0.213350,-0.021550
3,-1.702315,-0.531236,1.358662,-0.081037,-0.715003,-0.302266,-0.293333,-0.287823,-0.292522,-0.316258,-0.333484,-0.00147,0.213350,0.042870
4,-1.702315,-0.531236,-0.736018,-0.081037,-0.715003,3.308342,-0.264622,-0.281823,-0.292522,-0.323974,-0.333484,-0.00147,0.213350,-0.064139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5089970,3.333208,-0.531236,-0.736018,-0.081037,1.398595,-0.302266,-0.296288,-0.288791,-0.292522,-0.323974,-0.333484,-0.00147,0.211648,-0.017167
5089971,3.333208,-0.531236,1.358662,-0.081037,-0.715003,-0.302266,0.227730,-0.288736,-0.292522,-0.222373,-0.153113,-0.00147,-0.320453,-0.014652
5089972,3.333208,-0.531236,-0.736018,-0.081037,1.398595,-0.302266,-0.283673,-0.109086,-0.117972,-0.323974,-0.333484,-0.00147,0.213350,-0.035880
5089973,3.333208,-0.531236,1.358662,-0.081037,-0.715003,-0.302266,-0.034133,-0.287458,-0.292522,-0.323974,-0.290125,-0.00147,-0.048206,-0.014652


In [ ]:
train_label.to_frame()

,isFraud
0,1
1,1
2,1
3,1
4,1
...,...
5089970,0
5089971,0
5089972,0
5089973,0


In [ ]:
data_feature_test = pd.DataFrame(data=test_feature_scaled, columns=col)
data_feature_test

,step,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,errorBalanceOrig,errorBalanceDest
0,-1.702315,-0.531236,1.358662,-0.081037,-0.715003,-0.302266,0.391529,-0.288795,-0.292522,-0.323944,2.197310,-0.00147,-0.487147,21.806856
1,-1.702315,-0.531236,-0.736018,-0.081037,-0.715003,3.308342,1.818966,0.153620,-0.292522,-0.323974,-0.333484,-0.00147,0.213350,-3.154830
2,-1.702315,-0.531236,1.358662,-0.081037,-0.715003,-0.302266,1.818966,0.153620,-0.292522,-0.323974,0.332466,-0.00147,0.213350,2.856455
3,-1.702315,-0.531236,1.358662,-0.081037,-0.715003,-0.302266,-0.077800,-0.287236,-0.292522,-0.323974,-0.297301,-0.00147,-0.002766,-0.014652
4,-1.695292,-0.531236,-0.736018,-0.081037,-0.715003,3.308342,1.518921,0.090915,-0.292522,-0.323974,-0.333484,-0.00147,0.213350,-2.709757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1272640,3.333208,-0.531236,-0.736018,-0.081037,1.398595,-0.302266,-0.287422,-0.281319,-0.287318,-0.323974,-0.333484,-0.00147,0.213350,-0.030318
1272641,3.333208,-0.531236,-0.736018,-0.081037,1.398595,-0.302266,-0.287029,-0.278702,-0.284815,-0.323974,-0.333484,-0.00147,0.213350,-0.030902
1272642,3.333208,-0.531236,-0.736018,-0.081037,1.398595,-0.302266,-0.291172,-0.286882,-0.292039,-0.323974,-0.333484,-0.00147,0.213350,-0.024755
1272643,3.333208,-0.531236,1.358662,-0.081037,-0.715003,-0.302266,0.010761,-0.230585,-0.292522,-0.316644,-0.275967,-0.00147,0.182656,-0.014652


In [ ]:
test_label.to_frame()

,isFraud
0,1
1,1
2,1
3,1
4,1
...,...
1272640,0
1272641,0
1272642,0
1272643,0


## Logistic Regression

Logistic regression is used to describe data and to explain the relationship between one dependent binary variable and one or more nominal, ordinal, interval or ratio-level independent variables
    
    class sklearn.linear_model.LogisticRegression (penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
    
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0)
clf.fit(train_feature_scaled,train_label)

CPU times: user 1min 54s, sys: 16.4 s, total: 2min 11s
Wall time: 1min 25s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
# predict train data
lr_predY_train = clf.predict(train_feature_scaled)

In [ ]:
lr_predY_train

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
# predict test data
lr_predY = clf.predict(test_feature_scaled)
lr_predY[0:10]

array([0, 1, 0, 0, 1, 0, 1, 0, 0, 0])

## XGBoost 

XGBoost stands for Extreme Gradient Boosting; it is a specific implementation of the Gradient Boosting method which uses more accurate approximations to find the best tree model. It employs a number of nifty tricks that make it exceptionally successful, particularly with structured data.

XGBoost's hyperparameters

    -learning_rate: step size shrinkage used to prevent overfitting. Range is [0,1]
    -max_depth: determines how deeply each tree is allowed to grow during any boosting round.
    -subsample: percentage of samples used per tree. Low value can lead to underfitting.
    -colsample_bytree: percentage of features used per tree. High value can lead to overfitting.
    -n_estimators: number of trees you want to build.
    -objective: determines the loss function to be used like reg:linear for regression problems, reg:logistic for classification problems with only decision, binary:logistic for classification problems with probability.

XGBoost also supports regularization parameters to penalize models as they become more complex and reduce them to simple (parsimonious) models.

    -gamma: controls whether a given node will split based on the expected reduction in loss after the split. A higher value leads to fewer splits. Supported only for tree-based learners.
    -alpha: L1 regularization on leaf weights. A large value leads to more regularization.
    -lambda: L2 regularization on leaf weights and is smoother than L1 regularization.

In [ ]:
%%time
from xgboost.sklearn import XGBClassifier
xgbc = XGBClassifier(objective='binary:logistic',max_depth=3)


xgbc.fit(train_feature_scaled, train_label)

[02:16:49] WARNING: /workspace/src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
CPU times: user 14min 44s, sys: 1.44 s, total: 14min 46s
Wall time: 14min 42s


In [ ]:
 # predict train data
xgb_predY_train = xgbc.predict(train_feature_scaled)

In [ ]:
# predict test data
xgb_predY = xgbc.predict(test_feature_scaled)
xgb_predY[0:10]

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1])

## Model Evaluation

### Confusion Matrix

confusion matrix, also known as an error matrix, is a specific table layout that allows visualization of the performance of an algorithm, typically a supervised learning one (in unsupervised learning it is usually called a matching matrix). Each row of the matrix represents the instances in a predicted class while each column represents the instances in an actual class (or vice versa)



https://en.wikipedia.org/wiki/Confusion_matrix

--------------------------------------------
Accuracy: Overall, how often is the classifier correct?
    
    Accuracy = (TP+TN)/total = (TP+TN)/(TP+TN+FN+FP)

Precision: When it predicts yes, how often is it correct?

    Precision = TP/predicted yes = TP/(TP+FP)
    
Recall: When it's actually yes, how often does it predict yes? (True Positive Rate)

    Recall = TP/actual yes = TP/(TP+FN)
    
F1 Score: This is a weighted average of the recall and precision

    F1 Score = (2*(Precision*Recall))/(Precision+Recall)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

def evaluation_metrics(types,label_train,predict_train,label_test,predict_test):
  print("Accuracy score in",types, ": Train =" ,accuracy_score(label_train,predict_train), " , Test = ",accuracy_score(label_test,predict_test))
  print("Precision score in ",types, ": Train =" ,precision_score(label_train,predict_train), " , Test = ",precision_score(label_test,predict_test))
  print("Recall score in ",types, ": Train =",recall_score(label_train,predict_train), " , Test = ",recall_score(label_test,predict_test))
  print("F1 score  in ",types, ": Train =" ,f1_score(label_train,predict_train), " , Test = ",f1_score(label_test,predict_test))

In [ ]:
evaluation_metrics("Logistic Regression",train_label,lr_predY_train,test_label,lr_predY)

Accuracy score in Logistic Regression : Train = 0.9993573642306691  , Test =  0.9993611729901112
Precision score in  Logistic Regression : Train = 0.9241116751269035  , Test =  0.912041884816754
Recall score in  Logistic Regression : Train = 0.5505821866021473  , Test =  0.544375
F1 score  in  Logistic Regression : Train = 0.6900407467070975  , Test =  0.6818003913894326


In [ ]:
cm_lr = confusion_matrix(lr_predY,test_label,labels=[1,0])
cm_lr

array([[    871,      84],
       [    729, 1270961]])

In [ ]:
evaluation_metrics("XGBoost",train_label,xgb_predY_train,test_label,xgb_predY)

Accuracy score in XGBoost : Train = 0.9999943025260438  , Test =  0.9999937138793615
Precision score in  XGBoost : Train = 0.9993932038834952  , Test =  0.9987468671679198
Recall score in  XGBoost : Train = 0.9962195675185241  , Test =  0.99625
F1 score  in  XGBoost : Train = 0.9978038621734192  , Test =  0.9974968710888611


In [ ]:
cm_xgb = confusion_matrix(xgb_predY,test_label,labels=[1, 0])
cm_xgb

array([[   1594,       2],
       [      6, 1271043]])

## Random OverSampling 

Random oversampling involves randomly duplicating examples from the minority class and adding them to the training dataset.

This technique can be effective for those machine learning algorithms that are affected by a skewed distribution and where multiple duplicate examples for a given class can influence the fit of the model

In [ ]:
from imblearn.over_sampling import RandomOverSampler 
ros = RandomOverSampler()
train_feature_resampled, train_label_resampled = ros.fit_resample(train_feature, train_label)

In [ ]:
scaler = preprocessing.StandardScaler()
train_feature_scaled =scaler.fit_transform(train_feature_resampled)
test_feature_scaled = scaler.transform(test_feature)

In [ ]:
clf_ros = LogisticRegression(random_state=0)
clf_ros.fit(train_feature_scaled,train_label_resampled)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=0)

In [ ]:
lr_predY_ros_train = clf_ros.predict(train_feature_scaled)

In [ ]:
lr_predY_ros = clf_ros.predict(test_feature_scaled)


In [ ]:
evaluation_metrics("Logistic Regession after RandomOverSampling",train_label_resampled,lr_predY_ros_train,test_label,lr_predY_ros)

Accuracy score in Logistic Regession after RandomOverSampling : Train = 0.9669029079573715  , Test =  0.9521005464996131
Precision score in  Logistic Regession after RandomOverSampling : Train = 0.9539094149661581  , Test =  0.025196371722471963
Recall score in  Logistic Regession after RandomOverSampling : Train = 0.9812157780618418  , Test =  0.984375
F1 score  in  Logistic Regession after RandomOverSampling : Train = 0.9673699377991529  , Test =  0.04913506683928934


In [ ]:
cm_lr_ros = confusion_matrix(lr_predY_ros,test_label,labels=[1, 0])
cm_lr_ros

array([[   1575,   60934],
       [     25, 1210111]])

## Random Undersampling

Random undersampling involves randomly selecting examples from the majority class to delete from the training dataset.

This has the effect of reducing the number of examples in the majority class in the transformed version of the training dataset. This process can be repeated until the desired class distribution is achieved, such as an equal number of examples for each class

A limitation of undersampling is that examples from the majority class are deleted that may be useful, important, or perhaps critical to fitting a robust decision boundary. Given that examples are deleted randomly, there is no way to detect or preserve “good” or more information-rich examples from the majority class.

https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
train_feature_resampled, train_label_resampled = rus.fit_resample(train_feature,train_label)

In [ ]:
scaler = preprocessing.StandardScaler()
train_feature_scaled =scaler.fit_transform(train_feature_resampled)
test_feature_scaled = scaler.transform(test_feature)

In [ ]:
clf_rus = LogisticRegression(random_state=0)
clf_rus.fit(train_feature_scaled,train_label_resampled)

LogisticRegression(random_state=0)

In [ ]:
lr_predY_rus_train = clf_rus.predict(train_feature_scaled)

In [ ]:
lr_predY_rus = clf_rus.predict(test_feature_scaled)

In [ ]:
evaluation_metrics("Logistic Regression after RandomUnderSampling",train_label_resampled,lr_predY_rus_train,test_label,lr_predY_rus)

Accuracy score in Logistic Regression after RandomUnderSampling : Train = 0.9613639800393164  , Test =  0.9410071151027977
Precision score in  Logistic Regression after RandomUnderSampling : Train = 0.9429442508710801  , Test =  0.02052908395651436
Recall score in  Logistic Regression after RandomUnderSampling : Train = 0.9821563586874339  , Test =  0.983125
F1 score  in  Logistic Regression after RandomUnderSampling : Train = 0.9621509517813496  , Test =  0.04021835010163251


In [ ]:
cm_lr_rus = confusion_matrix(lr_predY_rus,test_label,labels=[1, 0])
cm_lr_rus

array([[   1573,   75050],
       [     27, 1195995]])